In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import json
import folium
from folium import plugins
import geopandas as gpd

In [3]:
rent_data = pd.read_csv('zillow-multifamily.csv')
rent_data.head(2)

FileNotFoundError: [Errno 2] File b'zillow-multifamily.csv' does not exist: b'zillow-multifamily.csv'

In [ ]:
rent_data["City"].value_counts()

In [ ]:
sf_rent = rent_data[rent_data["City"] == "San Francisco"]
nyc_rent = rent_data[rent_data["City"] == "New York"]

In [6]:
#bring in data
nyc_data = gpd.read_file('../data/nyc-merged.geojson')
nyc_data.drop(['id'], axis=1, inplace=True)

sf_data = gpd.read_file('../data/sf-multifamily-merged.geojson')
sf_data.drop(['id'], axis=1, inplace=True)

In [ ]:
sf_data.sort_values('Neighborhood', inplace=True)
sf_data.reset_index(inplace=True)
sf_data.drop(41, inplace=True)
sf_data[81::]["Neighborhood"]

In [ ]:
sf_data.head()

In [ ]:
#sf_rent.reset_index(inplace=True)
#sf_rent.drop(['level_0'], axis=1, inplace=True)
sf_rent.head()

In [ ]:
sf_rent.loc[41,"RegionName"] = "Northern Waterfront"

In [ ]:
sf_merged = pd.merge(sf_data, sf_rent[['RegionName','MedianRent']], left_on = 'Neighborhood', right_on = 'RegionName', how = 'outer')
sf_merged.tail()

In [ ]:
sf_merged.isnull().sum()

In [ ]:
sf_merged[sf_merged["Neighborhood"] == "Dolores Heights"]

In [ ]:
sf_rent.shape

In [ ]:
with open('sf-multifamily-merged.geojson', 'w') as f:
    f.write(sf_merged.to_json())

In [ ]:
plt.hist(sf_merged["Zillow Median"], alpha=0.5, label='All Homes + Multifamily')
plt.hist(sf_merged["MedianRent"], alpha=0.5, label='Multifamily')
plt.legend(loc='upper right')
plt.show()

In [ ]:
missing_in_region = []
missing_in_hoods = []
for h in sf_merged['Neighborhood'].to_list():
    if h not in sf_merged['RegionName'].to_list():
        missing_in_region.append(h)
        
for h in sf_merged['RegionName'].to_list():
    if h not in sf_merged['Neighborhood'].to_list():
        missing_in_hoods.append(h)
        
missing_in_region

In [ ]:
sf_data.shape

In [ ]:
#sf_rentcafe = pd.read_csv('sf-rentcafe.csv', delimiter='\t')
sf_rentcafe[70::]

In [ ]:
# sf_rentcafe.loc[24,"Neighborhood"] = "Parkside"
sf_rentcafe.loc[75,"Neighborhood"] = "Eureka Valley"

In [ ]:
#sf_merged = pd.merge(sf_merged, sf_rentcafe, left_on = 'Neighborhood', right_on = 'Neighborhood', how = 'outer')
sf_merged.tail(20)

In [ ]:
#sf_merged[sf_merged["Neighborhood"] == "Outer Richmond"]
sf_merged.loc[73,"Average Rent"] = float(3922)

In [ ]:
with open('sf-multifamily-merged.geojson', 'w') as f:
    f.write(sf_merged.to_json())

In [10]:
state_geo = f'../data/sf-multifamily-merged.geojson'
state_data = sf_data

m = folium.Map(location=[37.7749, -122.4194], zoom_start=12)
bins_multi = list(sf_data['MedianRent'].quantile([0, 0.25, 0.5, 0.75, 1]))

multi_choropleth = folium.Choropleth(
    geo_data=state_geo,
    name='Multifamily',
    data=sf_data,
    columns=['Neighborhood', 'MedianRent'],
    key_on='feature.properties.Neighborhood',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    bins=bins_multi,
    highlight=True,
    overlay=True,
    legend_name='Median Rent'
    
).add_to(m)

# folium.LayerControl().add_to(m)

multi_choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['Neighborhood','MedianRent'], aliases=['Neighborhood', 'Median Rent'])
)

bins_all = list(sf_data['Zillow Median'].quantile([0, 0.25, 0.5, 0.75, 1]))

all_choropleth = folium.Choropleth(
    geo_data=state_geo,
    name='All Homes + Multifamily',
    data=sf_data,
    columns=['Neighborhood', 'Zillow Median'],
    key_on='feature.properties.Neighborhood',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    bins=bins_all,
    highlight=True,
    overlay=True,
    legend_name='ZRI Median'
    
).add_to(m)

#folium.LayerControl().add_to(m)

all_choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['Neighborhood','Zillow Median'], aliases=['Neighborhood', 'Median Rent']))

bins_avg = list(sf_data['Average Rent'].quantile([0, 0.25, 0.5, 0.75, 1]))

avg_choropleth = folium.Choropleth(
    geo_data=state_geo,
    name='Average Rent',
    data=sf_data,
    columns=['Neighborhood', 'Average Rent'],
    key_on='feature.properties.Neighborhood',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    bins=bins_avg,
    highlight=True,
    overlay=True,
    legend_name='Average Rent'
    
).add_to(m)

folium.LayerControl().add_to(m)

avg_choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['Neighborhood','Average Rent'], aliases=['Neighborhood', 'Average Rent']))

m